In [24]:
import openai

In [25]:
import os, dotenv
dotenv.load_dotenv(r'C:\Users\i.berros.sanmartin\Documents\MIO\EjemploAI\.env')

# Setup environment
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_MODEL = os.getenv("AZURE_OPENAI_MODEL")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

# Libraries
from openai import AzureOpenAI

In [26]:
AZURE_OPENAI_EMBEDDING = os.getenv("AZURE_OPENAI_EMBEDDING")
print(AZURE_OPENAI_EMBEDDING)

text-embedding-ada-002


In [27]:
client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

### Conexion Azure OpenAI

In [28]:
# Hacer una consulta al modelo
response = client.chat.completions.create(
    model=AZURE_OPENAI_DEPLOYMENT_NAME,  # Nombre del despliegue, no el modelo
    messages=[{"role": "system", "content": "Eres un asistente útil."},
              {"role": "user", "content": "¿Cuál es la capital de España?"}]
)

# Imprimir la respuesta
print(response.choices[0].message.content)

La capital de España es Madrid.


### Conexion con langchain

In [33]:
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
import os


# Inicializar el modelo de Azure OpenAI
llm = AzureChatOpenAI(
    azure_deployment=AZURE_OPENAI_DEPLOYMENT_NAME,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    temperature=0,
    max_retries=2

)

# Prueba básica del modelo
def test_llm():
    messages = [
        SystemMessage(content="Eres un asistente útil."),
        HumanMessage(content="¿Cuál es la capital de España?")
    ]
    response = llm(messages)
    print(response.content)

test_llm()


La capital de España es Madrid.


# Agent Job Search

In [ ]:
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
import os
from playwright.sync_api import sync_playwright
from bs4 import BeautifulSoup


# Job Finder Agent
def job_finder(search_query):
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        page = browser.new_page()
        
        url = f"https://www.linkedin.com/jobs/search?keywords={search_query}"
        page.goto(url)
        
        html_content = page.content()
        browser.close()
        
    soup = BeautifulSoup(html_content, "html.parser")
    jobs = soup.find_all("div", class_="job-card-container")
    
    job_list = []
    for job in jobs[:5]:  # Extraer solo los primeros 5
        title = job.find("h3").text if job.find("h3") else "N/A"
        company = job.find("h4").text if job.find("h4") else "N/A"
        link = job.find("a")["href"] if job.find("a") else "#"
        job_list.append({"title": title, "company": company, "link": link})
    
    return job_list

# Prueba del Job Finder
if __name__ == "__main__":
    jobs = job_finder("Data Scientist")
    for job in jobs:
        print(job)


# Agent highlights Job

In [42]:
job_text_example = """EY wavespace Madrid is looking for a data scientist with background in Generative AI technologies to help design and build cutting-edge Gen AI solutions. EY wavespace is creating innovative AI technology and products that will transform EY and our clients. We are creating a model for how AI can reinvent large companies and industries. Through EY’s network of more than 250,000 professionals working in every sector, we have an opportunity to offer AI products and services that transform how business is done in all types of enterprises and to realize the vision of building a better working world.

    Responsibilities:
    - Lead the design, development, and implementation of state-of-the-art Generative AI models to automate tax-related processes and improve efficiency.
    - Collaborate closely with cross-functional teams to understand business requirements and challenges.
    - Stay up to date with the latest advancements in Generative AI.
    - Optimize and fine-tune existing Generative AI models to enhance their performance and accuracy.
    - Provide technical guidance and support to junior members of the team, promoting knowledge sharing.
    - Communicate complex concepts and findings to both technical and non-technical stakeholders through documentation and presentations.
    - Stay informed about industry trends and emerging technologies related to Generative AI.

    Requirements:
    - Master’s in Computer Science, Data Science, AI, or any related field.
    - 3-5 years of hands-on experience in developing and deploying AI models, preferably in finance or tax domain.
    - Experience developing complex software systems that have been deployed to an active userbase.
    - Strong understanding of Machine Learning models.
    - Experience in Natural Language Processing.
    - Proven track record of successfully delivering complex Generative AI projects and generating actionable insights from large-scale datasets.
    - Excellent problem-solving skills, with the ability to think creatively and strategically to develop innovative solutions.
    - Strong leadership and communication skills, with the ability to effectively collaborate with cross-functional teams and present complex concepts to stakeholders.
    - A portfolio showcasing successful projects or research papers is highly desirable.
    
    If you feel that you meet these requirements, please apply with your ENGLISH CV.
    """

In [43]:
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
import os

# Initialize Azure OpenAI model
llm = AzureChatOpenAI(
    azure_deployment=AZURE_OPENAI_DEPLOYMENT_NAME,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    temperature=0,
    max_retries=2
)

# Agent to extract key points from a job description
def extract_key_points(job_description):
    prompt = f"""
    From the following job description, extract the key points to highlight in a CV:
    - List 5 most important skills required for the job.
    - List 5 key technologies mentioned.
    - Identify the 2 most important programming languages mentioned.
    Return the information as three separate bullet-point lists.
    
    Job description:
    {job_description}
    """
    messages = [
        SystemMessage(content="You are an expert assistant in helping candidates optimize their CVs."),
        HumanMessage(content=prompt)
    ]
    response = llm(messages)
    return response.content

# Test the key points extractor agent
if __name__ == "__main__":
    job_text = job_text_example
    
    key_points = extract_key_points(job_text)
    print(key_points)


**Key Skills Required:**
- Generative AI model design and development
- Collaboration with cross-functional teams
- Optimization and fine-tuning of AI models
- Technical guidance and mentorship
- Strong communication and presentation skills

**Key Technologies Mentioned:**
- Generative AI
- Machine Learning
- Natural Language Processing (NLP)
- AI products and services
- Data analysis and insights generation

**Most Important Programming Languages:**
- Python
- R


# Agent to read CV 

In [44]:
pdf_path_example = r"C:\Users\i.berros.sanmartin\Documents\MIO\EjemploAI\Irene_CV\CV_template.pdf"

In [38]:
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
import os
import fitz  # PyMuPDF for reading PDFs

# Initialize Azure OpenAI model
llm = AzureChatOpenAI(
    azure_deployment=AZURE_OPENAI_DEPLOYMENT_NAME,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    temperature=0,
    max_retries=2
)

# Function to extract text from a PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

# Agent to compare CV with job requirements and suggest improvements
def optimize_cv(cv_text, job_key_points):
    prompt = f"""
    Given the following extracted CV text and the key points from a job description,
    suggest specific improvements the candidate should make to better align with the role.
    
    CV Text:
    {cv_text}
    
    Job Key Points:
    {job_key_points}
    
    Provide actionable recommendations for missing skills, technologies, and key experience areas.
    """
    messages = [
        SystemMessage(content="You are an expert assistant in optimizing CVs for job applications."),
        HumanMessage(content=prompt)
    ]
    response = llm(messages)
    return response.content



In [45]:
# Test the CV Optimizer Agent
if __name__ == "__main__":
    # Path to the sample CV PDF
    pdf_path = pdf_path_example  # Replace with actual file path
    
    # Extract text from the CV
    cv_text = extract_text_from_pdf(pdf_path)
    
    # Sample job key points (from previous agent output)
    job_key_points = key_points
    
    # Get suggestions for improvement
    improvements = optimize_cv(cv_text, job_key_points)
    print(improvements)


To better align Erik Duthiers' CV with the job description provided, here are specific recommendations for improvements:

### 1. **Highlight Generative AI Experience**
   - **Action**: Emphasize the experience with generative AI in the CV. The project descriptions should explicitly mention the design and development aspects of generative AI models.
   - **Example Improvement**: In the "Sales Coach Assistant for BBVA Gen AI" project, add details about the model architecture used, any specific algorithms implemented, and the outcomes of the project in terms of performance metrics.

### 2. **Emphasize Optimization and Fine-Tuning Skills**
   - **Action**: Include specific examples of how Erik has optimized or fine-tuned AI models in previous projects.
   - **Example Improvement**: Add a bullet point under relevant projects that describes any techniques used for model optimization, such as hyperparameter tuning, feature selection, or performance evaluation.

### 3. **Technical Guidance and

# Agents chain

In [51]:
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
import os
import fitz  # PyMuPDF for reading PDFs

# Initialize Azure OpenAI model
llm = AzureChatOpenAI(
    azure_deployment=AZURE_OPENAI_DEPLOYMENT_NAME,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    temperature=0,
    max_retries=2
)

# Agent to extract key points from a job description
def extract_key_points(job_description):
    prompt = f"""
    From the following job description, extract the key points to highlight in a CV:
    - List 5 most important skills required for the job.
    - List 5 key technologies mentioned.
    - Identify the 2 most important programming languages mentioned.
    Return the information as three separate bullet-point lists.
    
    Job description:
    {job_description}
    """
    messages = [
        SystemMessage(content="You are an expert assistant in helping candidates optimize their CVs."),
        HumanMessage(content=prompt)
    ]
    response = llm(messages)
    return response.content

# Function to extract text from a PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

# Agent to compare CV with job requirements and suggest improvements
def optimize_cv(cv_text, job_key_points):
    prompt = f"""
    Given the following extracted CV text and the key points from a job description,
    provide structured recommendations to optimize the CV for the role:
    
    1. **Prioritization Order:** Outline the ideal structure and order of sections to emphasize based on the job requirements.
    2. **Must-Have Elements:** List the key skills, technologies, and experiences that must not be missing from the CV.
    3. **Suggested Improvements:** Recommend specific changes or additions to improve alignment with the job role.
    
    CV Text:
    {cv_text}
    
    Job Key Points:
    {job_key_points}
    """
    messages = [
        SystemMessage(content="You are an expert assistant in optimizing CVs for job applications."),
        HumanMessage(content=prompt)
    ]
    response = llm(messages)
    return response.content


# Chain of agents to process job description and CV
def process_cv_and_job(job_description, pdf_path):
    # Step 1: Extract key points from the job description
    job_key_points = extract_key_points(job_description)
    print("Extracted Job Key Points:\n", job_key_points)
    
    # Step 2: Extract text from the CV PDF
    cv_text = extract_text_from_pdf(pdf_path)
    print("Extracted CV Text:\n", cv_text[:500], "...")  # Print first 500 chars for preview
    
    # Step 3: Optimize the CV based on job key points
    improvements = optimize_cv(cv_text, job_key_points)
    print("CV Improvement Suggestions:\n", improvements)
    
    return improvements




In [52]:
# Test the agent chain
if __name__ == "__main__":
    # Sample job description
    job_text = job_text_example
    
    # Path to sample CV PDF (replace with actual path)
    pdf_path = pdf_path_example
    
    # Run the agent chain
    process_cv_and_job(job_text, pdf_path)

Extracted Job Key Points:
 **Key Skills Required:**
- Generative AI model design and development
- Collaboration with cross-functional teams
- Optimization and fine-tuning of AI models
- Technical guidance and mentorship
- Strong communication and presentation skills

**Key Technologies Mentioned:**
- Generative AI
- Machine Learning
- Natural Language Processing (NLP)
- AI products and services
- Data analysis and insights generation

**Most Important Programming Languages:**
- Python
- R
Extracted CV Text:
 Contributed to international AI projects across 3+ countries in Europe and the Middle East, working as
part of the European Artificial Intelligence Delivery Hub at Avanade.
Graduated MSc in Artificial Intelligence (with honors) from City, University of London. 
Awarded the Department of Computer Science Outstanding Project Prize for developing a Spanish
Poetry Generator from scratch.
Strong track record of cross-functional collaboration and delivering measurable results.
Endesa
Ga